In [1]:
from cassandra.cqlengine.models import Model
from cassandra.cqlengine import columns
from cassandra.cqlengine import connection
from cassandra.cqlengine.management import sync_table
from cassandra.cqlengine.query import BatchQuery

from app.lib.connections import *
from app.lib.models import *
from app.lib.helpers import *
from app.lib.utils import *

INFO : CONNECTING TO CASSANDRA
INFO : SUCCESS CONNECTION TO CASSANDRA


In [2]:
query = "SELECT keyspace_name FROM system_schema.keyspaces;"
ckeyspaces = connection.execute(query)
ckeyspaces = [keyspace["keyspace_name"] for keyspace in ckeyspaces]
ckeyspaces

['system_auth',
 'system_schema',
 'system_distributed',
 'system',
 'unilinkspace',
 'system_traces']

In [3]:
sync_table(UserCredentials)
sync_table(UserInfo)
sync_table(Posts)
sync_table(InterestedPosts)
sync_table(UserConnections)

c:\Users\Jashu\Desktop\unilink\backend\menvi\Lib\site-packages\cassandra\cqlengine\management.py:544: UserWarning: CQLENG_ALLOW_SCHEMA_MANAGEMENT environment variable is not set. Future versions of this package will require this variable to enable management functions.
  warnings.warn(msg)


In [3]:
class UserCredentials(Model):
    email = columns.Text(primary_key=True)
    hashed_password = columns.Text()

class UserInfo(Model):
    name = columns.Text(index=True)
    email = columns.Text(primary_key=True)

sync_table(UserCredentials)
sync_table(UserInfo)

c:\Users\Jashu\Desktop\unilink\backend\menvi\Lib\site-packages\cassandra\cqlengine\management.py:544: UserWarning: CQLENG_ALLOW_SCHEMA_MANAGEMENT environment variable is not set. Future versions of this package will require this variable to enable management functions.
  warnings.warn(msg)


In [3]:
values = UserCredentials.objects.all()
list(values)

[UserCredentials(email='mickey@gmail.com', hashed_password='$2b$12$EC.SUo/MLv72M0jfXe4WE.V7Kt6NdWabwC8ASLdRKBLRh4.O8lc4.'),
 UserCredentials(email='jake@gmail.com', hashed_password='$2b$12$kpfwTaygxWy6CiDftB6Qi.USflIDHcYOiCil3R8dT02uPURc0xQ6S'),
 UserCredentials(email='steve@gmail.com', hashed_password='$2b$12$KCOeo02uBnPPUUUdlMqZRuDnosoVk2s6x7pxZ6UaNXUM8jrnLVPS2'),
 UserCredentials(email='ash@gmail.com', hashed_password='$2b$12$hrTWdKsALg2JRcvTpSTEJucDoSMz0WAHQZ3nRDEWqSmILaPxErglC'),
 UserCredentials(email='peter@gmail.com', hashed_password='$2b$12$2g/YeD.1N4GskZGkEz/kMev25LfWegRqkQt8f4bUtBMEZEW2U.ZyG'),
 UserCredentials(email='james@gmail.com', hashed_password='$2b$12$Q.G5.CqDIGa/9qzxZN3I7OcIo.9z9VEYvEvoXEdF/ydggRsNMy2hK'),
 UserCredentials(email='alex@gmail.com', hashed_password='$2b$12$PwvVSaYJWsm/nwYNeaNZxOm/i2V4Bcxz8NcrP6n/vlGzV6WlXfiAy')]

In [4]:
class UserConnections(Model):
    from_email = columns.Text(primary_key=True)
    to_email = columns.Text(primary_key=True)
    from_name = columns.Text()
    to_name = columns.Text()

sync_table(UserConnections)

In [94]:
connection.execute("truncate table user_connections")

In [95]:
from_name = 'steve'
from_email = 'steve@gmail.com'

to_names = "mikey james ash peter".split()
for to_name in to_names : 
    to_email = f'{to_name}@gmail.com'
    data = {
        'from_email' : from_email,
        'from_name' : from_name,
        'to_email' : to_email,
        'to_name' : to_name,
    }
    UserConnections.create(**data)

In [96]:
from_names  = "user1 user2 user3".split()

to_email = 'steve@gmail.com'
to_name = 'steve'

for from_name in from_names:
    from_email = f'{from_name}@gmail.com'
    data = {
        'from_email' : from_email,
        'from_name' : from_name,
        'to_email' : to_email,
        'to_name' : to_name,
    }
    UserConnections.create(**data)

In [97]:
values = UserConnections.objects.all()
list(values)

[UserConnections(from_email='steve@gmail.com', to_email='ash@gmail.com', from_name='steve', to_name='ash'),
 UserConnections(from_email='steve@gmail.com', to_email='james@gmail.com', from_name='steve', to_name='james'),
 UserConnections(from_email='steve@gmail.com', to_email='mikey@gmail.com', from_name='steve', to_name='mikey'),
 UserConnections(from_email='steve@gmail.com', to_email='peter@gmail.com', from_name='steve', to_name='peter'),
 UserConnections(from_email='user2@gmail.com', to_email='steve@gmail.com', from_name='user2', to_name='steve'),
 UserConnections(from_email='user1@gmail.com', to_email='steve@gmail.com', from_name='user1', to_name='steve'),
 UserConnections(from_email='user3@gmail.com', to_email='steve@gmail.com', from_name='user3', to_name='steve')]

In [98]:
def get_connections(user_email):
    user_connections = list()

    from_connections = UserConnections.objects(from_email=user_email)
    to_connections = UserConnections.objects(to_email = user_email).allow_filtering()

    for iconnection in from_connections:
        user_connection = {'email' : iconnection.to_email,'name' : iconnection.to_name}
        user_connections.append(user_connection)
    for iconnection in to_connections:
        user_connection = {'email' : iconnection.from_email,'name' : iconnection.from_name}
        user_connections.append(user_connection)
    
    return user_connections
values = get_connections(user_email = 'steve@gmail.com')
values

[{'email': 'ash@gmail.com', 'name': 'ash'},
 {'email': 'james@gmail.com', 'name': 'james'},
 {'email': 'mikey@gmail.com', 'name': 'mikey'},
 {'email': 'peter@gmail.com', 'name': 'peter'},
 {'email': 'user2@gmail.com', 'name': 'user2'},
 {'email': 'user1@gmail.com', 'name': 'user1'},
 {'email': 'user3@gmail.com', 'name': 'user3'}]

In [99]:
auth_email = 'steve@gmail.com'
user_email = 'ash@gmail.com'
try:
    connection_to_delete = UserConnections.objects.get(
        from_email=user_email, to_email=auth_email
    )
    connection_to_delete.delete()
    print(f"Connection from {user_email} to {auth_email} deleted.")
except UserConnections.DoesNotExist:
    print(f"Connection from {user_email} to {auth_email} not found.")
except Exception as e:
    error_message = "INFO : /remove-connection : 1"
    print(error_message)

try:
    connection_to_delete = UserConnections.objects.get(
        from_email=auth_email, to_email=user_email
    )
    connection_to_delete.delete()
    print(f"Connection from {auth_email} to {user_email} deleted.")
except UserConnections.DoesNotExist:
    print(f"Connection from {auth_email} to {user_email} not found.")
except Exception as e:
    error_message = "INFO : /remove-connection : 2"
    print(error_message)

In [5]:
class Posts(Model):
    email = columns.Text(primary_key=True, partition_key=True)
    xtimestamp = columns.DateTime(
        primary_key=True, clustering_order="DESC", default=datetime.now
    )

    id = columns.UUID(primary_key=True, default=uuid.uuid4)
    title = columns.Text()
    description = columns.Text()
    interested = columns.Integer(default = 0)

sync_table(Posts)

In [4]:
values = Posts.objects.all()
list(values)

[Posts(email='jake@gmail.com', xtimestamp=datetime.datetime(2024, 4, 27, 14, 39, 56, 290000), id=UUID('540b6dd8-e569-4b58-89ba-0098b1d9e433'), title='dd', description='df', interested=0),
 Posts(email='jake@gmail.com', xtimestamp=datetime.datetime(2024, 4, 27, 14, 39, 8, 390000), id=UUID('78e07cf5-49b8-4bca-b84b-8d44aa88d202'), title='title2', description='desc2', interested=4),
 Posts(email='jake@gmail.com', xtimestamp=datetime.datetime(2024, 4, 27, 14, 19, 6, 827000), id=UUID('441cfab1-6bf5-4857-ac83-7d0aad345de8'), title='title3edit', description='desc3', interested=0),
 Posts(email='steve@gmail.com', xtimestamp=datetime.datetime(2024, 4, 26, 19, 36, 51, 746000), id=UUID('4468536c-d8ed-4bbf-92c9-292230b36fad'), title='title2', description='desc2', interested=1),
 Posts(email='steve@gmail.com', xtimestamp=datetime.datetime(2024, 4, 26, 19, 34, 37, 157000), id=UUID('77bbbee1-f5bd-43b4-beea-ab95c42f3f7c'), title='title one', description='description one', interested=0),
 Posts(email='j

In [ ]:
postid = '1fe1310c-7281-4675-a936-1e28bc066423'
Posts.objects.allow_filtering().get(id = postid).delete()

In [ ]:
class Likes(Model):
    email = columns.Text(primary_key=True, partition_key=True)
    xtimestamp = columns.DateTime(
        primary_key=True, clustering_order="DESC", default=datetime.now
    )

    email = columns.Text(primary_key=True, partition_key=True)
    id = columns.UUID(primary_key=True, default=uuid.uuid4)
    title = columns.Text()
    description = columns.Text()
    interested = columns.Integer(default = 0)

In [23]:
import random
import uuid
def create_random_post():
    emails = ['mickey@example.com', 'steve@example.com', 'ash@example.com']

    email = random.choice(emails)

    post = Posts(
        email=email,
        title='Random Post ' + str(uuid.uuid4()),
        description='This is a random post with ID ' + str(uuid.uuid4()),
    )

    post.save()

# for _ in range(10) : create_random_post()


AttributeError: 'Posts' object has no attribute 'allow_filtering'

In [12]:
values = Posts.objects.all()
list(values)

[Posts(email='jake@gmail.com', xtimestamp=datetime.datetime(2024, 4, 27, 14, 39, 56, 290000), id=UUID('540b6dd8-e569-4b58-89ba-0098b1d9e433'), title='dd', description='df', interested=0),
 Posts(email='jake@gmail.com', xtimestamp=datetime.datetime(2024, 4, 27, 14, 39, 8, 390000), id=UUID('78e07cf5-49b8-4bca-b84b-8d44aa88d202'), title='title2', description='desc2', interested=0),
 Posts(email='jake@gmail.com', xtimestamp=datetime.datetime(2024, 4, 27, 14, 19, 6, 827000), id=UUID('441cfab1-6bf5-4857-ac83-7d0aad345de8'), title='title3edit', description='desc3', interested=0),
 Posts(email='steve@gmail.com', xtimestamp=datetime.datetime(2024, 4, 26, 19, 36, 51, 746000), id=UUID('4468536c-d8ed-4bbf-92c9-292230b36fad'), title='title2', description='desc2', interested=0),
 Posts(email='steve@gmail.com', xtimestamp=datetime.datetime(2024, 4, 26, 19, 34, 37, 157000), id=UUID('77bbbee1-f5bd-43b4-beea-ab95c42f3f7c'), title='title one', description='description one', interested=0)]

In [10]:
title = 'titleEdited'
description = 'descEdited'

email = 'steve@gmail.com'
postid = '26c5fbca-9f4e-4458-8217-4cc613830d25'
post = Posts.objects.allow_filtering().get(id=postid)
post.title = title
post.description = description
post.save()

Posts(email='jake@gmail.com', xtimestamp=datetime.datetime(2024, 4, 27, 7, 58, 57, 946000), id=UUID('26c5fbca-9f4e-4458-8217-4cc613830d25'), title='titleEdited', description='descEdited', interested=0)

In [11]:
email = 'dd@gmail.com'

auth_posts = Posts.objects(email=email).all()

list(auth_posts)

[]

In [55]:
def get_connections(user_email):
    user_connections = list()

    from_connections = UserConnections.objects(from_email=user_email)
    to_connections = UserConnections.objects(to_email = user_email).allow_filtering()

    for iconnection in from_connections:
        user_connection = {'email' : iconnection.to_email,'name' : iconnection.to_name}
        user_connections.append(user_connection)
    for iconnection in to_connections:
        user_connection = {'email' : iconnection.from_email,'name' : iconnection.from_name}
        user_connections.append(user_connection)
    
    print(user_connection)

values = get_connections('steve@gmail.com')

{'email': 'user3@gmail.com', 'name': 'user3'}


In [ ]:
user_email = None
auth_email = None
def delete_connection(user_email, auth_email):
    try:
        connection_to_delete = UserConnections.objects.get(from_email=user_email, to_email=auth_email)
        connection_to_delete.delete()
        print(f"Connection from {user_email} to {auth_email} deleted.")
    except UserConnections.DoesNotExist:
        print(f"Connection from {user_email} to {auth_email} not found.")
    except Exception as e:
        # print(e)
        error_message = "INFO : /remove-connection : 2"
        print(error_message)

    try:
        connection_to_delete = UserConnections.objects.get(from_email=auth_email, to_email=user_email)
        connection_to_delete.delete()
        print(f"Connection from {auth_email} to {user_email} deleted.")
    except UserConnections.DoesNotExist:
        print(f"Connection from {auth_email} to {user_email} not found.")
    except Exception as e:
        # print(e)
        error_message = "INFO : /remove-connection : 2"
        print(error_message)


In [36]:
user_email = 'ash@gmail.com'
values = UserConnections.objects(from_email = user_email)
values = UserConnections.objects(to_email = user_email).allow_filtering()
list(values)

[UserConnections(from_email='steve@gmail.com', to_email='ash@gmail.com', from_name='steve', to_name='ash')]

In [6]:
class InterestedPosts(Model):
    email = columns.Text(primary_key=True, partition_key=True)
    post_id = columns.UUID(primary_key=True)
    xtimestamp = columns.DateTime(primary_key=True, clustering_order="DESC", default=datetime.now)

sync_table(InterestedPosts)

In [5]:
values = InterestedPosts.objects().all()
list(values)

[InterestedPosts(email='james@gmail.com', post_id=UUID('4468536c-d8ed-4bbf-92c9-292230b36fad'), xtimestamp=datetime.datetime(2024, 4, 28, 7, 17, 26, 629000)),
 InterestedPosts(email='alex@gmail.com', post_id=UUID('4468536c-d8ed-4bbf-92c9-292230b36fad'), xtimestamp=datetime.datetime(2024, 4, 28, 13, 42, 59, 588000)),
 InterestedPosts(email='alex@gmail.com', post_id=UUID('77bbbee1-f5bd-43b4-beea-ab95c42f3f7c'), xtimestamp=datetime.datetime(2024, 4, 28, 13, 43, 1, 658000))]

In [14]:
post_ids = ['78e07cf5-49b8-4bca-b84b-8d44aa88d202','441cfab1-6bf5-4857-ac83-7d0aad345de8','77bbbee1-f5bd-43b4-beea-ab95c42f3f7c']

def get_posts_by_ids(post_ids):
    posts = Posts.objects(id__in=post_ids).allow_filtering()
    return posts


In [15]:
values = get_posts_by_ids(post_ids)
list(values)

[Posts(email='jake@gmail.com', xtimestamp=datetime.datetime(2024, 4, 27, 14, 39, 8, 390000), id=UUID('78e07cf5-49b8-4bca-b84b-8d44aa88d202'), title='title2', description='desc2', interested=0),
 Posts(email='jake@gmail.com', xtimestamp=datetime.datetime(2024, 4, 27, 14, 19, 6, 827000), id=UUID('441cfab1-6bf5-4857-ac83-7d0aad345de8'), title='title3edit', description='desc3', interested=0),
 Posts(email='steve@gmail.com', xtimestamp=datetime.datetime(2024, 4, 26, 19, 34, 37, 157000), id=UUID('77bbbee1-f5bd-43b4-beea-ab95c42f3f7c'), title='title one', description='description one', interested=0)]

In [31]:
post = Posts.objects.allow_filtering().get(id='4468536c-d8ed-4bbf-92c9-292230b36fad')

In [49]:
values = Posts.objects(email = 'steve@gmail.com').all()

for value in list(values):
    print(value.title, value.description, value.interested)

title2 desc2 0
title one description one 0
